In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
pd.set_option('display.max_columns', 500)
from scipy import stats
from scipy.stats import norm,skew

In [ ]:
train = pd.read_csv('/kaggle/input/pubg-finish-placement-prediction/train_V2.csv')

In [ ]:
#test = pd.read_csv('/kaggle/input/pubg-finish-placement-prediction/test_V2.csv')

In [ ]:
train.shape #,test.shape

In [ ]:
train.isnull().sum()

In [ ]:
train.dropna(inplace=True)

In [ ]:
train.head()

groupId - Integer ID to identify a group within a match. If the same group of players plays in different matches, they will have a different groupId each time.

matchId - Integer ID to identify match. There are no matches that are in both the training and testing set.

assists - Number of enemy players this player damaged that were killed by teammates.(partial kills)

boosts - Number of boost items used.

damageDealt - Total damage dealt. Note: Self inflicted damage is subtracted.

DBNOs - Number of enemy players knocked.

headshotKills - Number of enemy players killed with headshots.

heals - Number of healing items used.

killPlace - Ranking in match of number of enemy players killed.

killPoints - Kills-based external ranking of player. (Think of this as an Elo ranking where only kills matter.)

kills - Number of enemy players killed.

killStreaks - Max number of enemy players killed in a short amount of time.

longestKill - Longest distance between player and player killed at time of death. This may be misleading, as downing a - player and driving away may lead to a large longestKill stat.

maxPlace - Worst placement we have data for in the match. This may not match with numGroups, as sometimes the data skips over placements.

numGroups - Number of groups we have data for in the match.

revives - Number of times this player revived teammates.

rideDistance - Total distance traveled in vehicles measured in meters.

roadKills - Number of kills while in a vehicle.

swimDistance - Total distance traveled by swimming measured in meters.

teamKills - Number of times this player killed a teammate.

vehicleDestroys - Number of vehicles destroyed.

walkDistance - Total distance traveled on foot measured in meters.

weaponsAcquired - Number of weapons picked up.

winPoints - Win-based external ranking of player. (Think of this as an Elo ranking where only winning matters.)

winPlacePerc - The target of prediction. This is a percentile winning placement, where 1 corresponds to 1st place, and 0 corresponds to last place in the match. It is calculated off of maxPlace, not numGroups, so it is possible to have missing chunks in a match.

In [ ]:
train['winPlacePerc'].value_counts()

In [ ]:
plt.figure(figsize=(15,8))
sns.distplot(train['winPlacePerc'],fit=norm)

In [ ]:
train['assists'].value_counts()

In [ ]:
plt.figure(figsize=(15,8))
sns.countplot(train['kills'].sort_values())

Most people do not even kill a single player So let's check do they do some damage

In [ ]:
plt.figure(figsize=(15,8))
sns.countplot(train['assists'].sort_values())

In [ ]:
for i in range(0,60):
    percentage = train[train['kills'] == i].shape[0]/len(train['kills']) * 100
    print("Number of players having",i,"kills in their matches:",percentage)

In [ ]:
plt.figure(figsize=(15,8))
sns.countplot(train['boosts'].sort_values())

In [ ]:
plt.figure(figsize=(15,8))
sns.countplot(train['assists'].sort_values())

There is minor difference between assists and boosts

Now checking the relation between number of kills and winPlacePerc

In [ ]:
plt.figure(figsize=(10,8))
sns.jointplot(x=train['winPlacePerc'],y=train['kills'])

In [ ]:
plt.figure(figsize=(10,8))
sns.jointplot(x=train['headshotKills'],y=train['kills'])

So as the number of headshots increases kills are also increasing

In [ ]:
plt.figure(figsize=(10,8))
sns.jointplot(x=train['killPlace'],y=train['kills'])

In [ ]:
plt.figure(figsize=(10,8))
sns.jointplot(x=train['killPoints'],y=train['kills'])

So most of the players having number of kills between 0-20 are getting more killPoints

In [ ]:
plt.figure(figsize=(10,8))
sns.countplot(train['killStreaks'])

In [ ]:
plt.figure(figsize=(10,8))
sns.distplot(train['walkDistance'],fit=norm)

In [ ]:
work_dist_zero = train[train['walkDistance'] <= 0].shape[0]
print('Number of people how walked zero distance that is they are killed or they exit the game before even stepping few steps',work_dist_zero)

In [ ]:
avg_work = np.average(train['walkDistance'])
print('Average distance worked by the player is',avg_work)

In [ ]:
plt.figure(figsize=(10,8))
sns.jointplot(x=train['winPlacePerc'],y=train['walkDistance'])

In [ ]:
plt.figure(figsize=(10,8))
sns.jointplot(x=train['winPlacePerc'],y=train['rideDistance'])

In [ ]:
avg_drive = np.average(train['rideDistance'])
print('Average distance worked by the player is',avg_drive)

In [ ]:
ride_dist_zero = train[train['rideDistance'] <= 0].shape[0]
print("Players how don't drive:",ride_dist_zero)

Let's check that destroying a vehicle and winning a match have some relation or not.....

In [ ]:
plt.figure(figsize=(10,8))
sns.pointplot(x=train['vehicleDestroys'],y=train['winPlacePerc'])

In [ ]:
data = train.copy()

In [ ]:
data = data.groupby('groupId')['kills'].sum().reset_index()

In [ ]:
data = data.sort_values(by='kills',ascending=False)

In [ ]:
data.head(50)

These are the top 50 groups that have killed max players in the match. Congrats them

In [ ]:
plt.figure(figsize=(15,8))
sns.jointplot(x=train['kills'],y=train['headshotKills'])

In [ ]:
plt.figure(figsize=(10,8))
sns.jointplot(x=train['winPlacePerc'],y=train['heals'])

So there is strong correlation between healing of the players and winPlace

Now let's look at the hero's of the match which give revive to other people

In [ ]:
plt.figure(figsize=(10,8))
sns.jointplot(x=train['winPlacePerc'],y=train['revives'])

Now let's look at one of the important aspect i.e whether the player is solo or duo or group

In [ ]:
solos = train[train['numGroups']>50]
duos = train[(train['numGroups']>25) & (train['numGroups']<=50)]
squads = train[train['numGroups']<=25]

In [ ]:
print('they were around',(solos.shape[0]/train.shape[0]) * 100,'% of the solo games')
print('they were around',(duos.shape[0]/train.shape[0]) * 100,'% of the duos games')
print('they were around',(squads.shape[0]/train.shape[0]) * 100,'% of the squads games')

In [ ]:
f,ax1 = plt.subplots(figsize =(20,10))
sns.pointplot(x='kills',y='winPlacePerc',data=solos,color='black',alpha=0.8)
sns.pointplot(x='kills',y='winPlacePerc',data=duos,color='#CC0000',alpha=0.8)
sns.pointplot(x='kills',y='winPlacePerc',data=squads,color='#3399FF',alpha=0.8)
plt.text(37,0.6,'Solos',color='black',fontsize = 17,style = 'italic')
plt.text(37,0.55,'Duos',color='#CC0000',fontsize = 17,style = 'italic')
plt.text(37,0.5,'Squads',color='#3399FF',fontsize = 17,style = 'italic')
plt.xlabel('Number of kills',fontsize = 15,color='blue')
plt.ylabel('Win Percentage',fontsize = 15,color='blue')
plt.title('Solo vs Duo vs Squad Kills',fontsize = 20,color='blue')
plt.grid()
plt.show()

In [ ]:
f,ax = plt.subplots(figsize=(15, 15))
train_matrix = train.corr()

sns.heatmap(train_matrix,annot=True)

In [ ]:
def heatmap(x, y, size):
    fig, ax = plt.subplots()
    
    # Mapping from column names to integer coordinates
    x_labels = [v for v in sorted(x.unique())]
    y_labels = [v for v in sorted(y.unique())]
    x_to_num = {p[1]:p[0] for p in enumerate(x_labels)} 
    y_to_num = {p[1]:p[0] for p in enumerate(y_labels)} 
    
    size_scale = 500
    ax.scatter(
        x=x.map(x_to_num), # Use mapping for x
        y=y.map(y_to_num), # Use mapping for y
        s=size * size_scale, # Vector of square sizes, proportional to size parameter
        marker='s' # Use square as scatterplot marker
    )
    
    # Show column labels on the axes
    ax.set_xticks([x_to_num[v] for v in x_labels])
    ax.set_xticklabels(x_labels, rotation=45, horizontalalignment='right')
    ax.set_yticks([y_to_num[v] for v in y_labels])
    ax.set_yticklabels(y_labels)

In [ ]:
train_matrix['winPlacePerc'].sort_values(ascending=False)

In [ ]:
columns = ['walkDistance','boosts','weaponsAcquired','damageDealt','heals','kills','longestKill']

corr = train[columns].corr()
corr = pd.melt(corr.reset_index(), id_vars='index') # Unpivot the dataframe, so we can get pair of arrays for x and y
corr.columns = ['x', 'y', 'value']
heatmap(
    x=corr['x'],
    y=corr['y'],
    size=corr['value'].abs()
)

In [ ]:
train.columns

In [ ]:
train.drop(['Id','groupId','matchId'],axis=1,inplace=True)

In [ ]:
train.head()

In [ ]:
train['matchType'].value_counts()

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le = LabelEncoder()

In [ ]:
train['matchType'] = le.fit_transform(train['matchType'])

In [ ]:
train['matchType'].astype('int32')

In [ ]:
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.ensemble import RandomForestRegressor
import xgboost
from xgboost import XGBRegressor

In [ ]:
features = train.drop('winPlacePerc',axis=1)
labels = train['winPlacePerc']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(features,labels,test_size=0.1,random_state=42,shuffle=True)

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
ss = StandardScaler()

In [ ]:
X_train_scaled = ss.fit_transform(X_train)
X_test_scaled = ss.transform(X_test)

In [ ]:
X_train_scaled

In [ ]:
X_test_scaled

In [ ]:
np.max(X_test_scaled),np.max(X_train_scaled)

In [ ]:
lr = LinearRegression()

In [ ]:
lr.fit(X_train_scaled,y_train)

In [ ]:
ypred_lr = lr.predict(X_test_scaled)

In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
mean_absolute_error(ypred_lr,y_test)

In [ ]:
xgb = XGBRegressor()

In [ ]:
lasso = Lasso()

In [ ]:
lasso.fit(X_train_scaled,y_train)

In [ ]:
ypred_lasso = lasso.predict(X_test_scaled)

In [ ]:
mean_absolute_error(ypred_lasso,y_test)